## Preparaciones previas

In [163]:
from IPython.display import display, Markdown, Latex
from pulp import *
import numpy as np
import pandas as pd
import io

In [164]:
sub = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

## Definición del problema

In [165]:
n_of_workers = 8

n_of_shifts = 4

n_of_months = 2
n_of_weeks = 4 * n_of_months
version = 2

In [166]:
W = np.array([[1, 0.5, 0.25, 0],
              [1, 0.25, 0.5, 0],
              [1, 0.5, 0.25, 0],
              [1, 0.5, 0.25, 0],
              [0.5, 0.25, 1, 0],
              [0.25, 0.5, 1, 0],
              [0.5, 0.25, 1, 0],
              [0, 0, 0, 0]])

problem = LpProblem("max_problem", LpMaximize)

deltas = np.array(
    [[[LpVariable("\u03B4" + (str(i) + "," + str(j) + "," + str(k)).translate(sub), cat="Binary") 
     for k in range(n_of_weeks)]
     for j in range(n_of_shifts)]
     for i in range(n_of_workers)])

deltas_weight = np.array(
    [[[deltas[i,j,k] * W[i,j] 
     for k in range(n_of_weeks)]
     for j in range(n_of_shifts)]
     for i in range(n_of_workers)])

### Función objetivo: versión 1

In [167]:
#problem += lpSum(deltas_weight)

### Función objetivo: versión 2

In [168]:
H = np.array(
    [[LpVariable("H" + (str(i) + "," + str(j)).translate(sub)) 
                    for j in range(n_of_workers)] 
                    for i in range(n_of_workers)])
h = np.array(
    [[LpVariable("h" + (str(i) + "," + str(j)).translate(sub)) 
                    for j in range(n_of_workers)] 
                    for i in range(n_of_workers)])

happiness = np.sum(deltas_weight, axis=(2,1))/n_of_weeks
happiness_diff = np.array([[happiness[i] - happiness[j] for j in range(n_of_workers)] for i in range(n_of_workers)])

for i in range(n_of_workers):
    for j in range(n_of_workers):
        problem += H[i,j] == happiness_diff[i,j]
        problem += H[i,j] <= h[i,j]
        problem += -H[i,j] <= h[i,j]
        problem += 0 <= h[i,j]

problem += lpSum(deltas_weight) - lpSum(h)

### Restricciones

Restricción 1: al menos dos trabajadores por turno cada semana

In [169]:
for j in range(n_of_shifts):
    for k in range(n_of_weeks):
        problem += lpSum(deltas[:,j,k]) == 2

Restricción 2: un trabajador no puede hacer dos turnos en una misma semana

In [170]:
for i in range(n_of_workers):
    for k in range(n_of_weeks):
        problem += lpSum(deltas[i,:,k]) == 1

Restricción 3: un trabajador en concreto, denotado como $t$, solo puede hacer los turnos contenidos en el conjunto $R$

In [171]:
def rest3(problem, t, R):
    for k in range(n_of_weeks):
        for j in R:
            problem += deltas[t,j,k] <= 1
        for j in {i for i in range(n_of_shifts)} - R:
            problem += deltas[t,j,k] <= 0

rest3(problem, 7, {2,3})

Restricción 4: el tercer turno lo tiene que hacer cada trabajador una vez al mes

In [172]:
for i in range(n_of_workers):
    problem += lpSum(deltas[i,3,:]) == int(n_of_weeks/4)

Restricción 5: un trabajador solo puede hacer el tercer turno con tres semanas de  $$\begin{gather*}\delta_{i,3,k}\le 1-\delta_{i,3,k-q}\space\land\space \delta_{i,3,k-4}\le\delta_{i,3,k}\\\space\forall\space i=0,\dots,n\space\forall\space k=4,\dots,p\space\forall\space q=1,2,3\end{gather*}$$

In [173]:
for i in range(n_of_workers):
    if k < 4:
        break
    for k in range(4,n_of_weeks):
        problem += deltas[i,3,k-4] <= deltas[i,3,k]
        problem += deltas[i,3,k] <= 1 - deltas[i,3,k-3]
        problem += deltas[i,3,k] <= 1 - deltas[i,3,k-2]
        problem += deltas[i,3,k] <= 1 - deltas[i,3,k-1]

## Solución

In [174]:
problem.solve(PULP_CBC_CMD(msg=False))
print("\nPROBLEM STATUS\n" + str(LpStatus[problem.status]))


PROBLEM STATUS
Optimal


In [175]:
header = "| Trabajador |"
separator_elem = "--|"
separator = "|" + separator_elem
for k in range(n_of_weeks):
    header += " Semana " + str(k) + " |"
    separator += separator_elem

shift_list = ["Turno de mañana", "Turno de mediodía", "Turno de tarde", "Turno de fin de semana"]
table = ""
for i in range(n_of_workers):
    table += "| Trabajador " + str(i) + " |"
    for k in range(n_of_weeks):
        this_deltas = [deltas[i,j,k].value() for j in range(n_of_shifts)]
        shift = this_deltas.index(1)
        table += " " + shift_list[shift] + " |"
    table += "\n"

table = header + "\n" + separator + "\n" + table
display(Markdown(table))

| Trabajador | Semana 0 | Semana 1 | Semana 2 | Semana 3 | Semana 4 | Semana 5 | Semana 6 | Semana 7 |
|--|--|--|--|--|--|--|--|--|
| Trabajador 0 | Turno de mediodía | Turno de fin de semana | Turno de mediodía | Turno de mañana | Turno de mediodía | Turno de fin de semana | Turno de mañana | Turno de mañana |
| Trabajador 1 | Turno de fin de semana | Turno de mañana | Turno de mañana | Turno de mediodía | Turno de fin de semana | Turno de mañana | Turno de mañana | Turno de mediodía |
| Trabajador 2 | Turno de mañana | Turno de mañana | Turno de mediodía | Turno de fin de semana | Turno de mañana | Turno de mediodía | Turno de mediodía | Turno de fin de semana |
| Trabajador 3 | Turno de fin de semana | Turno de mediodía | Turno de mañana | Turno de mediodía | Turno de fin de semana | Turno de mañana | Turno de mediodía | Turno de mañana |
| Trabajador 4 | Turno de mañana | Turno de fin de semana | Turno de tarde | Turno de mañana | Turno de mañana | Turno de fin de semana | Turno de tarde | Turno de tarde |
| Trabajador 5 | Turno de mediodía | Turno de mediodía | Turno de fin de semana | Turno de tarde | Turno de tarde | Turno de tarde | Turno de fin de semana | Turno de mediodía |
| Trabajador 6 | Turno de tarde | Turno de tarde | Turno de tarde | Turno de fin de semana | Turno de mediodía | Turno de mediodía | Turno de tarde | Turno de fin de semana |
| Trabajador 7 | Turno de tarde | Turno de tarde | Turno de fin de semana | Turno de tarde | Turno de tarde | Turno de tarde | Turno de fin de semana | Turno de tarde |


In [176]:
header = "Trabajador"
for k in range(n_of_weeks):
    header += ",Semana " + str(k)
    separator += separator_elem

shift_list = ["Turno de mañana", "Turno de mediodía", "Turno de tarde", "Turno de fin de semana"]
table = ""
for i in range(n_of_workers):
    table += "Trabajador " + str(i)
    for k in range(n_of_weeks):
        this_deltas = [deltas[i,j,k].value() for j in range(n_of_shifts)]
        shift = this_deltas.index(1)
        table += "," + shift_list[shift]
    table += "\n"

table = header + "\n" + table
table_df = pd.read_csv(io.StringIO(table), sep=",")
table_df.to_excel("turnos.xlsx")